https://github.com/burnash/gspread

spreadsheet to process: https://docs.google.com/spreadsheets/d/1B0qccrlgnUDuNzyadA4pWvjDJs3XuHzkFVX1en8C2Eg/edit#gid=0

In [1]:
import gspread
import json

g_name = "Gitenberg Book List (Summer 2015) || Contributors: Jennifer 8. Lee and Michelle Cheng"
url = "https://docs.google.com/spreadsheets/d/1B0qccrlgnUDuNzyadA4pWvjDJs3XuHzkFVX1en8C2Eg/edit#gid=0"
g_key = "1B0qccrlgnUDuNzyadA4pWvjDJs3XuHzkFVX1en8C2Eg"


# OAuth2: Method 1

[Using OAuth2 for Authorization — gspread 0.2.5 documentation](http://gspread.readthedocs.org/en/latest/oauth2.html)

Created: https://console.developers.google.com/project/nypl50-gspread/apiui/credential#

```
pip install --upgrade oauth2client
```

I'd have to share the spreadsheet with `535523918532-5ejclnn335tr2g1u0dqnvh7g39q78mim@developer.gserviceaccount.com` -- so let's look at method 2

In [2]:
import json
import gspread
from oauth2client.client import SignedJwtAssertionCredentials

json_key = json.load(open('nypl50-gspread.json'))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)

gc = gspread.authorize(credentials)

#wks = gc.open_by_key(g_key).sheet1
wks = gc.open(g_name).sheet1

# OAuth2:  Method 2 -- actually do the auth

https://developers.google.com/api-client-library/python/guide/aaa_oauth and
https://developers.google.com/api-client-library/python/auth/installed-app

In [ ]:
credentials = json.loads(open("client_secret.json").read())
(credentials["installed"]["client_id"], credentials["installed"]["client_secret"])

In [ ]:
from oauth2client import client
import webbrowser

flow = client.flow_from_clientsecrets(
    'client_secret.json',
    scope='https://spreadsheets.google.com/feeds',
    redirect_uri='urn:ietf:wg:oauth:2.0:oob')

In [ ]:
auth_uri = flow.step1_get_authorize_url()
webbrowser.open_new(auth_uri)

In [ ]:
auth_code = raw_input("Some input please: ")
credentials = flow.step2_exchange(auth_code)
credentials

In [ ]:
gc = gspread.authorize(credentials)
wks = gc.open_by_url(url).sheet1

# Calculations

In [3]:
wks

<Worksheet 'RTC 100 (or 98)' id:od6>

In [4]:
all_rows = wks.get_all_values()

In [5]:
import pandas as pd
from pandas import (DataFrame, Series)

In [6]:
df = DataFrame(all_rows[2:], columns=all_rows[1])
df.index = df.index + 3  # shift index to match spreadsheet
df.head()

,Title,Author,Gitenberg Status,Gitenberg URL,Repo Cover,RTC 1st GEN,RTC 2nd GEN,RTC Other Gen,RTC Cover?,RTC Good Cover?,CAN REQUEST DATE,Notes,name in rtc/books.json
3,Adventures of Huckleberry Finn,Mark Twain,Needs Check,https://github.com/GITenberg/Adventures-of-Huc...,YES,X,,,YES,,,,The Adventures of Huckleberry Finn
4,The Ingenious Gentleman Don Quixote of La Mancha,Miguel de Cervantes,ASCIIDOC,https://github.com/GITenberg/Don-Quixote_996,YES,X,,,YES,,,,Don Quixote
5,Dubliners,James Joyce,"Needs Build, Needs Cover",https://github.com/GITenberg/Dubliners_2814,,X,,,YES,,,,Dubliners
6,Jane Eyre,Charlotte Bronte,Needs Build,https://github.com/GITenberg/Jane-Eyre_1260,YES,X,,,YES,,,,Jane Eyre
7,Moby-Dick,Herman Melville,Needs Build,https://github.com/GITenberg/Moby-Dick--Or-The...,YES,X,,,YES,,,,Moby Dick


In [7]:
# what does the status mean?

df[df["RTC Cover?"] == 'YES']["Gitenberg Status"].value_counts()

ASCIIDOC                    33
Needs Build                  7
                             6
Needs Check                  2
Needs Build, Needs Cover     1
CHECK                        1
dtype: int64

In [8]:
rtc50 = df[df["RTC Cover?"] == 'YES']
rtc50.head()

,Title,Author,Gitenberg Status,Gitenberg URL,Repo Cover,RTC 1st GEN,RTC 2nd GEN,RTC Other Gen,RTC Cover?,RTC Good Cover?,CAN REQUEST DATE,Notes,name in rtc/books.json
3,Adventures of Huckleberry Finn,Mark Twain,Needs Check,https://github.com/GITenberg/Adventures-of-Huc...,YES,X,,,YES,,,,The Adventures of Huckleberry Finn
4,The Ingenious Gentleman Don Quixote of La Mancha,Miguel de Cervantes,ASCIIDOC,https://github.com/GITenberg/Don-Quixote_996,YES,X,,,YES,,,,Don Quixote
5,Dubliners,James Joyce,"Needs Build, Needs Cover",https://github.com/GITenberg/Dubliners_2814,,X,,,YES,,,,Dubliners
6,Jane Eyre,Charlotte Bronte,Needs Build,https://github.com/GITenberg/Jane-Eyre_1260,YES,X,,,YES,,,,Jane Eyre
7,Moby-Dick,Herman Melville,Needs Build,https://github.com/GITenberg/Moby-Dick--Or-The...,YES,X,,,YES,,,,Moby Dick


# rtc covers


https://raw.githubusercontent.com/plympton/rtc/master/books.json

In [9]:
import requests
rtc_covers_url = "https://raw.githubusercontent.com/plympton/rtc/master/books.json"
covers = requests.get(rtc_covers_url).json()
covers_dict = dict([(cover['name'], cover) for cover in covers])

In [29]:
# read off cover_map from df
# http://stackoverflow.com/a/9762084
cover_map = dict(filter(lambda (k,v):v,
                  [tuple(x) for x in  df[['Title', 'name in rtc/books.json']].values]
            ))

In [30]:
from IPython.display import HTML
from PIL import Image
import jinja2

In [31]:
HTML("<b>hi</b>")

In [35]:
# let's look at the images for the books
# https://cdn.rawgit.com/plympton/rtc/master/rtc_books/
# https://cdn.rawgit.com/plympton/rtc/master/rtc_books_resized/

cover_url_base = "https://cdn.rawgit.com/plympton/rtc/master/rtc_books/"
small_cover_url_base = "https://cdn.rawgit.com/plympton/rtc/master/rtc_books_resized/"

from functools import partial

def cover_name_to_url(name, reduce=False):
    
    if reduce:
        url = small_cover_url_base
    else:
        url = cover_url_base
        
    cover = covers_dict.get(name)
    if cover is not None:
        return url + cover['covers'][0]["filename"]
    else:
        return None
    
cover_name_to_url_small = partial(cover_name_to_url, reduce=True)
cover_name_to_url_big = partial(cover_name_to_url, reduce=False)


rtc50 = df[df["RTC Cover?"] == 'YES']
df['image_url'] = rtc50['name in rtc/books.json'].map(cover_name_to_url_small)

In [37]:
rtc50 = df[df["RTC Cover?"] == 'YES']
rtc50.head()

,Title,Author,Gitenberg Status,Gitenberg URL,Repo Cover,RTC 1st GEN,RTC 2nd GEN,RTC Other Gen,RTC Cover?,RTC Good Cover?,CAN REQUEST DATE,Notes,name in rtc/books.json,image_url
3,Adventures of Huckleberry Finn,Mark Twain,Needs Check,https://github.com/GITenberg/Adventures-of-Huc...,YES,X,,,YES,,,,The Adventures of Huckleberry Finn,https://cdn.rawgit.com/plympton/rtc/master/rtc...
4,The Ingenious Gentleman Don Quixote of La Mancha,Miguel de Cervantes,ASCIIDOC,https://github.com/GITenberg/Don-Quixote_996,YES,X,,,YES,,,,Don Quixote,https://cdn.rawgit.com/plympton/rtc/master/rtc...
5,Dubliners,James Joyce,"Needs Build, Needs Cover",https://github.com/GITenberg/Dubliners_2814,,X,,,YES,,,,Dubliners,https://cdn.rawgit.com/plympton/rtc/master/rtc...
6,Jane Eyre,Charlotte Bronte,Needs Build,https://github.com/GITenberg/Jane-Eyre_1260,YES,X,,,YES,,,,Jane Eyre,https://cdn.rawgit.com/plympton/rtc/master/rtc...
7,Moby-Dick,Herman Melville,Needs Build,https://github.com/GITenberg/Moby-Dick--Or-The...,YES,X,,,YES,,,,Moby Dick,https://cdn.rawgit.com/plympton/rtc/master/rtc...


In [ ]:
im = Image.open('Foto.jpg')
im.save('Foto.png')

In [43]:
results = rtc50[['Title', 'image_url']].T.to_dict().values()

In [47]:
from IPython.display import HTML
from jinja2 import Template

CSS = """
<style>
  .wrap img {
    margin-left: 0px;
    margin-right: 0px;
    display: inline-block;
    width: 100px;
  }
</style>
"""

IMAGES_TEMPLATE = CSS + """
<div class="wrap">
 {% for item in items %}<img title="{{item.Title}}" src="{{item.image_url}}"/>{% endfor %}
</div>
"""
    
template = Template(IMAGES_TEMPLATE)
HTML(template.render(items=results)) 

In [54]:
#let's try looping over all the images and convert them to png 

import requests
from itertools import islice

img_path = "/Users/raymondyee/Downloads/rtc/"
for image in islice(results,None):
    url = image['image_url']
    if url is not None:
        name = url.split("/")[-1]
        content = requests.get(url).content
        with open(img_path + name, "wb") as f:
            f.write(content)

In [61]:
f = "/Users/raymondyee/Downloads/rtc/rtc_A+Tale+of+Two+Cities_Alexis+Lampley.jpg"
".".join(f.split(".")[:-1]) + ".png"

'/Users/raymondyee/Downloads/rtc/rtc_A+Tale+of+Two+Cities_Alexis+Lampley.png'

In [65]:
# loop over jpg and convert to png

import glob

for f in glob.glob("/Users/raymondyee/Downloads/rtc/*.jp*g"):
    im = Image.open(f)
    png_path = ".".join(f.split(".")[:-1]) + ".png"
    if im.mode not in ["1", "L", "P", "RGB", "RGBA"]:
        im = im.convert("RGB")
    im.save(png_path)

In [84]:
[cover['covers'][0]['filename'] for cover in covers]

[u'rtc_Call+of+the+Wild_Michael+van+Kekem.jpeg',
 u'rtc_The+Brothers+Karamazov_Roberlan+Borges.jpeg',
 u'rtc_The+Jungle_Wade+Greenberg.jpeg',
 u'rtc_Ulysses_nick+fairbank+-+f9+design.jpeg',
 u'rtc_Heart+of+Darkness_Louise+Norman.jpeg',
 u'rtc_The+Metamorphosis_Frank+Myles.jpeg',
 u'rtc_Three+Musketeers_Ioannis+Fetanis.jpeg',
 u'rtc_The+Good+Soldier_vikram+nongmaithem.jpeg',
 u'rtc_Flatland_alex+mckeithen.jpeg',
 u'rtc_War+of+the+Worlds_Kjell+Roger+Ringstad.jpeg',
 u'rtc_Twenty+Thousand+Leagues+Under+the+Sea_Wade+Greenberg.jpeg',
 u'rtc_Crime+and+Punishment_Mario+Fuentes.jpeg',
 u'rtc_The+Jungle+Book_Jeff+Walters.jpeg',
 u'rtc_Little+Women_Lia+Marcoux.jpeg',
 u'rtc_Grimm+s+Fairy+Tales_Kathryn+Delaney.jpeg',
 u'rtc_Swann+s+Way_Louise+Norman.jpeg',
 u'rtc_Pride+and+Prejudice_Tim+Daniel.jpeg',
 u'rtc_War+and+Peace_T++M+++Serlin.jpeg',
 u'rtc_Moby+Dick_Black+Dwarf+Designs.png',
 u'rtc_The+Scarlet+Letter_MrFurious.jpeg',
 u'rtc_Persuasion_Heather+Main.png',
 u'rtc_Dr+++Jekyll+and+Mr+++Hyde_m

In [77]:
# image types in covers
from collections import Counter
map(lambda p: p.split(".")[-1], reduce(lambda x,y: x+y, [[c['filename'] for c in cover['covers'] for cover in covers]]))

[u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg',
 u'jpg']